In [2]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Define the path to the directory
directory_path = r'C:\Users\shaif\Downloads\Compressed\Invasive_Life_Stage'

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for folder in os.listdir(directory_path):
    #print(class_folder)
    clsp = os.path.join(directory_path,folder)  
    for class_folder in os.listdir(clsp):
        #print(class_folder)
        class_path = os.path.join(clsp, class_folder)    
        #print(class_path)
        for image_file in os.listdir(class_path):
            if image_file.endswith('.png'):
                image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
                img = Image.open(image_path).convert('RGB')
                img = img.resize((32, 32))
                img_array = np.array(img)
            
            # Append image and label to lists
                X_train.append(img_array)
                Y_train.append(class_label)
                #print(class_label)
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')


X_train shape: (49227, 32, 32, 3)
Y_train shape: (49227,)


In [3]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Y_train_encoded = to_categorical(Y_train)
# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train_encoded, test_size=0.2, random_state=42)

In [5]:
from tensorflow import keras
from keras.models import load_model
# Load the model
model_path = r"C:\Users\shaif\Downloads\mob_invasive_ssl.h5"
model = load_model(model_path)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling1D

num_classes = 10  # Update with the actual number of classes in your target data
model.layers[0].trainable = False
x = model.layers[-1].output  # Access the last 4th layer from the end
output = Dense(5, activation='softmax')(x)

# Create the new model with the updated head
new_model = keras.models.Model(inputs=model.input, outputs=output)
# Compile the model
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
new_model.fit(np.array(X_train).astype('float32'),y_train, batch_size=32, epochs=50, validation_split = .2)

Epoch 1/50
985/985 [==============================] - 56s 45ms/step - loss: 332.7286 - accuracy: 0.3231 - val_loss: 338.5182 - val_accuracy: 0.3986
Epoch 2/50
985/985 [==============================] - 49s 50ms/step - loss: 59.8942 - accuracy: 0.3297 - val_loss: 122.0125 - val_accuracy: 0.4082
Epoch 3/50
985/985 [==============================] - 46s 46ms/step - loss: 22.2506 - accuracy: 0.3293 - val_loss: 42.0655 - val_accuracy: 0.4504
Epoch 4/50
985/985 [==============================] - 48s 49ms/step - loss: 13.9325 - accuracy: 0.3463 - val_loss: 16.9711 - val_accuracy: 0.3943
Epoch 5/50
985/985 [==============================] - 45s 45ms/step - loss: 8.1732 - accuracy: 0.3781 - val_loss: 8.2128 - val_accuracy: 0.3301
Epoch 6/50
985/985 [==============================] - 55s 56ms/step - loss: 6.2079 - accuracy: 0.3973 - val_loss: 1.9638 - val_accuracy: 0.3694
Epoch 7/50
985/985 [==============================] - 60s 61ms/step - loss: 4.6245 - accuracy: 0.4062 - val_loss: 2.4652 - va

In [8]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
predictions = new_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
balanced_accuracy = balanced_accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Balanced Accuracy:", balanced_accuracy)
print("Test Accuracy:", accuracy)

Test Balanced Accuracy: 0.3626704844512506
Test Accuracy: 0.5712979890310786


In [11]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load MobileNetV2 model without top layer
base_model = MobileNetV2(include_top=False, weights="imagenet", input_shape=(32,32,3), pooling="avg")

# Add custom top layers
output = Dense(5, activation='softmax')(base_model.output)  # Update num_classes with your number of classes

# Create the new model
new_model = Model(inputs=base_model.input, outputs=output)

# Compile the model
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
new_model.fit(X_train, y_train, batch_size=32, epochs=50, validation_split=0.2)


Epoch 1/50
985/985 [==============================] - 38s 34ms/step - loss: 0.7399 - accuracy: 0.7307 - val_loss: 2.0733 - val_accuracy: 0.5714
Epoch 2/50
985/985 [==============================] - 32s 32ms/step - loss: 0.7220 - accuracy: 0.7454 - val_loss: 2.0845 - val_accuracy: 0.5842
Epoch 3/50
985/985 [==============================] - 37s 38ms/step - loss: 0.6633 - accuracy: 0.7623 - val_loss: 4.0186 - val_accuracy: 0.3425
Epoch 4/50
985/985 [==============================] - 38s 39ms/step - loss: 0.8076 - accuracy: 0.7163 - val_loss: 5.9666 - val_accuracy: 0.4996
Epoch 5/50
985/985 [==============================] - 33s 34ms/step - loss: 0.6769 - accuracy: 0.7529 - val_loss: 0.8721 - val_accuracy: 0.7664
Epoch 6/50
985/985 [==============================] - 32s 33ms/step - loss: 0.5615 - accuracy: 0.7961 - val_loss: 0.7575 - val_accuracy: 0.7408
Epoch 7/50
985/985 [==============================] - 31s 31ms/step - loss: 0.5085 - accuracy: 0.8165 - val_loss: 0.5625 - val_accuracy:

In [12]:
from sklearn.metrics import accuracy_score
predictions = new_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Accuracy:", accuracy)

Test Accuracy: 0.8384115376802762


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load MobileNetV2 model without top layer
base_model = MobileNetV2(include_top=False, weights=None, input_shape=(32,32,3), pooling="avg")

# Add custom top layers
output = Dense(5, activation='softmax')(base_model.output)  # Update num_classes with your number of classes

# Create the new model
new_model = Model(inputs=base_model.input, outputs=output)

# Compile the model
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
new_model.fit(X_train, y_train, batch_size=32, epochs=50, validation_split=0.2)


In [ ]:
from sklearn.metrics import accuracy_score
predictions = new_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Accuracy:", accuracy)